# Transposition Cipher Crack

## Import modules

In [214]:
from math import log
from itertools import permutations

## My Functions

In [215]:
# change rows to cols
def rows(s, n):
    rows = []
    for i in range(0,len(s),n):
        rows.append(s[i:i+n])
    return rows
def row2col(s, n): 
    cols = []
    for i in range(n):
        cols.append('')
    for i in range(len(s)):
        cols[i % n] += s[i]
    return cols

## Using sbox to transposition columns
def transcols(cols, box):
    trans = []
    for sb in box:
        trans.append(cols[sb])
    return trans

# encryption
# encrypt plaintext 
def enc(p, box):
    c = ''
    trans = transcols(row2col(p, len(box)), box)
    for i in range(len(p) / len(box)):
        for t in trans:
            c += t[i]
    ci = ''
    for t in trans:
        ci += t
    return ci 

# decryption
# find reverse shift box and decrypt ciphertext
def dec(c, box):
    p = ''
    trans = transcols(rows(c,int(len(c)/len(box))), box)
    for i in range(int(len(c) / len(box))):
        for t in trans:
            p += t[i]
    return p



# Analysis ngram-frequency 
def ngram_frequency(n, s):
    elements = []
    for i in range(len(s)-n+1):
        element = s[i:i+n]
        elements.append(element)
    d = {}
    for e in sorted(set(elements)):
        d[e] = s.count(e)
    return d

def W_32(s):
    tri_dist = ngram_frequency(3, s)
    di_dist = ngram_frequency(2, s)

    magic_num = 26 
    result = '### 3_2 ###\n\n trigram     digram   W \n---------------------------------------------\n'
    l = {}
    for t,f in tri_dist.items():
        pc = (float(f) / di_dist[t[:2]]) * magic_num
        result += '  ' + t + ' ' + str(f).rjust(3) + '    ' + t[:2] + ' ' + str(di_dist[t[:2]]).rjust(3) + '   ' + str(log(pc,10)) + '\n'
        l[t] = log(pc,10)
    print (result)
    return l

def W_21(s):
    di_dist = ngram_frequency(2, s)
    mono_dist = ngram_frequency(1, s)

    magic_num = 26 
    result = '### 2_1 ###\n\n digram     monogram   W \n---------------------------------------------\n'
    l = {}
    for d,f in di_dist.items():
        pc = (float(f) / mono_dist[d[:1]]) * magic_num
        result += '  ' + d + ' ' + str(f).rjust(3) + '    ' + d[:1] + ' ' + str(mono_dist[d[:1]]).rjust(4) + '   ' + str(log(pc,10)) + '\n'
        l[d] = log(pc,10)
    print (result)
    return l

# vowel analysis cipher
def count_vowel(str):
    vowel = 'AEIOU'
    sum = 0
    for v in vowel:
        sum += str.count(v)
    return sum

def get_factor(no):
    factors = []
    for i in range(5,int(no ** 0.5)):
        if no % i == 0:
            factors.append(i)
            factors.append(int(no/i))
    return sorted(factors)
def standard_deviation(col,exp,cipher):
    result = 0
    for i in range(0,len(cipher),col):
        result += (count_vowel(cipher[i:i+col])-exp)**2 
    row = len(cipher)/col
    return result/row

def result(col,cipher):
    exp = 0.4 * col
    result = standard_deviation(col,exp,cipher)
    cipher_matrix = ''.join(' ' + cipher[i:i+col] + ' ' + str(count_vowel(cipher[i:i+col])).rjust(5) + ' ' + str((count_vowel(cipher[i:i+col])-exp)**2).rjust(11) + '\n' for i in range(0,len(cipher),col))
    print (str(int(len(cipher)/col)) + ' x ' + str(col) + ' : ')
    print (' cipher'.ljust(col+2) + 'vowel'.rjust(6) + '  difference(count-expective & square)')
    print ('-' * 50)
    print (cipher_matrix)
    print (' standard_deviation of ' + str(int(len(cipher)/col)) + ' x ' + str(col) + ' = ' + str(result) + '\n\n')
    return result

# Compare original plaintext and cracked plaintext
def rate(s1, s2):
    correct = len(s1) # or len(s2)
    for x,y in zip(s1,s2):
        if x != y:
            correct -= 1
    rate = correct / float(len(s1)) * 100
    return rate

## Crack 

### step 1. get ciphertext

In [216]:
# Read plain
f = open('plain','r')
plain = f.read()
f.close()
plain = plain.replace('\n','')
plain = plain.upper()
print ('Plaintext = \n' + plain)

# shift box
sbox = [0, 1, 2, 3, 18, 17, 16, 4, 5, 6, 7, 10, 9, 8, 11, 12, 13, 14, 15]
print ('-' * 100)
print ('Shift = ' + str(sbox))
print ('-' * 100)

# Read cipher
f = open('cipher','r')
cipher = f.read()
f.close()
cipher = cipher.replace(' ','')
print ('Ciphertext = \n' + cipher)

Plaintext = 
REPRESENTATIVESANDDIRECTTAXESSHALLBEAPPORTIONEDAMONGTHESEVERALSTATESWHICHMAYBEINCLUDEDWITHINTHISUNIONACCORDINGTOTHEIRRESPECTIVENUMBERSWHICHSHALLBEDETERMINEDBYADDINGTOTHEWHOLENUMBEROFFREEPERSONSINCLUDINGTHOSEBOUNDTOSERVICEFORTHEM
----------------------------------------------------------------------------------------------------
Shift = [0, 1, 2, 3, 18, 17, 16, 4, 5, 6, 7, 10, 9, 8, 11, 12, 13, 14, 15]
----------------------------------------------------------------------------------------------------
Ciphertext = 
RIPVBSESIOOUEROEEUIWNLNNPERRINRHEESDRCTANIRIDNITDPEYIHRMHSOMDASAHTERWRBENEEMTOBEEEEHETILCOECBUNOSTOSLNSHYMCSEANTUAPSABLENXEADCEHDEURTSMSWRILNFNCASAEDOTLIOIITEDTECCADRDVIHOWIDVBGFGEVANHTIEETRTFELGIHNNDOEHOSLTCIGUETEORABHHNTMTHPST


### step 2. Training

In [217]:
model = open('model','r')
m = model.read()
# remove all nochar
for c in m:
    if (ord(c) > 122 or ord(c) < 97) and (ord(c) > 90 or ord(c) < 65):
        m = m.replace(c,'')
# change all char to upper
m = m.upper()
d_32 = W_32(m)
d_21 = W_21(m)

### 3_2 ###

 trigram     digram   W 
---------------------------------------------
  ABI   2    AB   4   1.1139433523068367
  ABR   2    AB   4   1.1139433523068367
  ACC   1    AC   3   0.9378520932511554
  ACH   2    AC   3   1.2388820889151366
  AGA   2    AG   5   1.0170333392987803
  AGE   3    AG   5   1.1931245983544614
  AID   2    AI   6   0.9378520932511554
  AIM   2    AI   6   0.9378520932511554
  AIN   2    AI   6   0.9378520932511554
  AKE   2    AK   2   1.414973347970818
  ALA   1    AL  24   0.03476210625921191
  ALE   3    AL  24   0.5118833609788743
  ALI   2    AL  24   0.3357921019231931
  ALL  14    AL  24   1.18089014193745
  ALO   2    AL  24   0.3357921019231931
  ALP   1    AL  24   0.03476210625921191
  ALS   1    AL  24   0.03476210625921191
  AME   3    AM   4   1.2900346113625178
  AMO   1    AM   4   0.8129133566428555
  ANC   1    AN  30   -0.062147906748844454
  AND   9    AN  30   0.8920946026904804
  ANE   1    AN  30   -0.062147906748844454
  ANO   

### step 3. Vowel analysis to get the most possible numbers of columns

In [218]:
def vowel_analysis(s):
    print ('Executing vowel analysis...\n\n')
    factors = get_factor(len(s))
    minsd = 10 
    for f in factors: # f is the possible number of columns
        c = ''
        col = row2col(s, int(len(s)/f)) # string len / factor must be integer (because it is factor)
        for cl in col:
            c += cl
        sd = result(f,c)  # give possible number of columns(f) & ciphertext(c) get standard derivation 
        if sd < minsd:    # if the sd minimum, f is the ideal number of columns
            minsd = sd
            ideal_col = f 
    print ('\nBy the Vowel-Analysis above,\nI consider ' + str(int(len(s)/ideal_col)) + ' x ' + str(ideal_col) + ' is the better way to split the ciphertext ~~\n')
    return ideal_col
idealcol = vowel_analysis(cipher)

Executing vowel analysis...


38 x 6 : 
 cipher  vowel  difference(count-expective & square)
--------------------------------------------------
 RTTPIT     1 1.960000000000001
 IAOSOF     4 2.5599999999999987
 PNBAIE     3 0.3599999999999996
 VIEBIL     3 0.3599999999999996
 BRELTG     1 1.960000000000001
 SIEEEI     5 6.759999999999998
 EDENDH     2 0.16000000000000028
 SNHXTN     0 5.760000000000002
 IIEEEN     5 6.759999999999998
 OTTACD     2 0.16000000000000028
 ODIDCO     3 0.3599999999999996
 UPLCAE     3 0.3599999999999996
 EECEDH     3 0.3599999999999996
 RYOHRO     2 0.16000000000000028
 OIEDDS     3 0.3599999999999996
 EHCEVL     2 0.16000000000000028
 ERBUIT     3 0.3599999999999996
 UMURHC     2 0.16000000000000028
 IHNTOI     3 0.3599999999999996
 WSOSWG     1 1.960000000000001
 NOSMIU     3 0.3599999999999996
 LMTSDE     1 1.960000000000001
 NDOWVT     1 1.960000000000001
 NASRBE     2 0.16000000000000028
 PSLIGO     2 0.16000000000000028
 EANLFR     2 0.1600000000000002

### step 4. Try the first two cols (if I don't know first two cols)

In [219]:
columns = rows(cipher, int(len(cipher)/idealcol))
print ('\ncipher rows <=> cols\n')
print (' elements in columns\n')
for s in columns:
    print ('col ' + str(columns.index(s)).rjust(2) + ' ' + s)
pool = []
for i in range(19):
    pool.append(i)
first2 = {}
for index in permutations(pool,2): # try every permutations of {0, 1, 2,..., 19} get two
    wsum = 0
    for a,b in zip(columns[ index[0] ], columns[ index[1] ]):
        d_gram = a + b
        if d_gram in d_21:
            wsum += d_21[d_gram]
    # find some possible pairs that can be first two columns (by statictis)
    if wsum > 3:
        first2[index] = wsum

print ('\nShow high freguency column pairs: \n')
for k,v in first2.items():
    # Show high freguency column pairs
    print (k, v)


cipher rows <=> cols

 elements in columns

col  0 RIPVBSESIOOU
col  1 EROEEUIWNLNN
col  2 PERRINRHEESD
col  3 RCTANIRIDNIT
col  4 DPEYIHRMHSOM
col  5 DASAHTERWRBE
col  6 NEEMTOBEEEEH
col  7 ETILCOECBUNO
col  8 STOSLNSHYMCS
col  9 EANTUAPSABLE
col 10 NXEADCEHDEUR
col 11 TSMSWRILNFNC
col 12 ASAEDOTLIOII
col 13 TEDTECCADRDV
col 14 IHOWIDVBGFGE
col 15 VANHTIEETRTF
col 16 ELGIHNNDOEHO
col 17 SLTCIGUETEOR
col 18 ABHHNTMTHPST

Show high freguency column pairs: 

(0, 1) 5.205851311174599
(1, 2) 5.281214423197684
(1, 8) 4.375128905740397
(2, 3) 4.641119454193328
(2, 12) 4.394249492945526
(3, 0) 3.6822574453920813
(3, 6) 3.295932362395615
(3, 7) 5.80093058503662
(3, 12) 3.9613178780030656
(4, 1) 3.0945543108848694
(4, 7) 3.696443562083942
(4, 9) 3.176166702455541
(4, 15) 3.376944724123847
(5, 17) 4.368479991995517
(6, 5) 5.583294916814542
(7, 2) 4.382789542606682
(7, 8) 3.5314453600813795
(7, 15) 3.5658172338735192
(7, 17) 4.2605397638737195
(8, 9) 3.257244922387166
(8, 15) 4.030475706699414
(

In [220]:
# wsum is used to sum the w of one column set
# wwsum is used to sum the wsum of each wsum of the whole article

# Find max wwsum of whole article, I can get the most possible first two columns
# and use the first two columns to init the reverse shift box

print ('\nShow the wwsum(Wsum) of whole artile vs. cracked rate: \n')
Wmax = 0 
for k,v in first2.items():
    rsbox = [k[0], k[1]]
    notinbox = []
    for i in range(19):
        notinbox.append(i)
    for r in rsbox:
        notinbox.remove(r)

    wwsum = 0 # wsum of whole article
    while len(notinbox) != 0:
        wsum = -10
        for i in notinbox:
            sum = 0
            for a,b,c in zip(columns[rsbox[-2]], columns[rsbox[-1]], columns[i]):
                t_gram = a + b + c
                if t_gram in d_32:
                    sum += d_32[t_gram]
            if sum > wsum:
                maxi = i
                wsum = sum
        rsbox.append(maxi)
        notinbox.remove(maxi)
        wwsum += wsum
    print ('Wsum = ' + str(wwsum).rjust(20) + '  ;Rate = ' + (str(rate(plain, dec(cipher,rsbox))) + '%').rjust(25))
    if wwsum > Wmax:
        Wmax = wwsum
        ideal_rsbox_init = rsbox


Show the wwsum(Wsum) of whole artile vs. cracked rate: 

Wsum =   148.35790311460178  ;Rate =                    100.0%
Wsum =   146.19203101932192  ;Rate =        4.824561403508771%
Wsum =     100.045586213587  ;Rate =       3.9473684210526314%
Wsum =   141.28540940558872  ;Rate =        5.263157894736842%
Wsum =   101.54962768830107  ;Rate =       3.9473684210526314%
Wsum =   110.04583496125116  ;Rate =        5.263157894736842%
Wsum =    64.47266613070968  ;Rate =        3.070175438596491%
Wsum =   136.24942772459806  ;Rate =        5.263157894736842%
Wsum =    92.40712270629396  ;Rate =       3.9473684210526314%
Wsum =   142.56103494029222  ;Rate =        90.35087719298247%
Wsum =    75.87626903317272  ;Rate =        6.578947368421052%
Wsum =    82.46399221476382  ;Rate =       3.9473684210526314%
Wsum =    74.83456989622172  ;Rate =        6.578947368421052%
Wsum =   109.26678910896644  ;Rate =       60.526315789473685%
Wsum =    100.6230006899431  ;Rate =       10.52631578947368

### step 5. Now I know the init columns and I can use trigram analysis dataset to crack the ciphertext 

In [221]:
ideal_rsbox = ideal_rsbox_init
notinbox = []
for i in range(0,19):
    notinbox.append(i)
for r in ideal_rsbox:
    notinbox.remove(r)
while len(notinbox) != 0: 
    wsum = 0
    for i in notinbox:
        sum = 0
        for a,b,c in zip(columns[ideal_rsbox[-2]], columns[ideal_rsbox[-1]], columns[i]):
            t_gram = a + b + c
            if t_gram in d_32:
                sum += d_32[t_gram]
        if sum > wsum:
            maxi = i
            wsum = sum
    ideal_rsbox.append(maxi)
    notinbox.remove(maxi)
print ('\nreverse shift box = ' + str(ideal_rsbox))
print ('\nPlaintext = \n' + plain)
print ('\nCracked cipher (' + str(rate(plain, dec(cipher,ideal_rsbox))) + '%) = \n' + dec(cipher,ideal_rsbox))


reverse shift box = [0, 1, 2, 3, 7, 8, 9, 10, 13, 12, 11, 14, 15, 16, 17, 18, 6, 5, 4]

Plaintext = 
REPRESENTATIVESANDDIRECTTAXESSHALLBEAPPORTIONEDAMONGTHESEVERALSTATESWHICHMAYBEINCLUDEDWITHINTHISUNIONACCORDINGTOTHEIRRESPECTIVENUMBERSWHICHSHALLBEDETERMINEDBYADDINGTOTHEWHOLENUMBEROFFREEPERSONSINCLUDINGTHOSEBOUNDTOSERVICEFORTHEM

Cracked cipher (100.0%) = 
REPRESENTATIVESANDDIRECTTAXESSHALLBEAPPORTIONEDAMONGTHESEVERALSTATESWHICHMAYBEINCLUDEDWITHINTHISUNIONACCORDINGTOTHEIRRESPECTIVENUMBERSWHICHSHALLBEDETERMINEDBYADDINGTOTHEWHOLENUMBEROFFREEPERSONSINCLUDINGTHOSEBOUNDTOSERVICEFORTHEM
